In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences


# Place dataset path here
path = "../Datasets/CoAID/"


dfTrue = pd.read_csv(path+"NewsRealCOVID-19.csv", usecols=['title'], nrows=500)
dfTrue['label']=1
dfFalse = pd.read_csv(path+"NewsFakeCOVID-19.csv", usecols=['title'], nrows=500)
dfFalse['label']=0
dfTotal = pd.concat([dfTrue, dfFalse])

X = dfTotal['title'].values
y = dfTotal['label'].values

X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(
  X, y, test_size=0.25, random_state=450)



path_external = "/content/drive/MyDrive/CoVerifi&MedVerifi/" # Replace with appropriate path
validationNewsDF = pd.read_csv(path_external+"Combined_News.csv", usecols=['title', 'label'])


def binaryLabel(label):
  if label == "TRUE":
    return 1
  return 0

validationNewsDF['label'] = validationNewsDF['label'].apply(lambda label: binaryLabel(str(label)))

validationX = validationNewsDF['title'].values
validationX = [one_hot(str(elem),10000) for elem in validationX]
validationX = pad_sequences(validationX, padding='post', maxlen=500)
validationY = validationNewsDF['label'].values

In [6]:
#For SVM, LR, and NB, we used the default settings provided in the scikit-learn 
#For CNN we use the standard implementation with default setting https://github.com/dennybritz/cnn-text-classification-tf
#Could just use an out-of-the-box CNN implementation from a TF tutorial
# The dennybritz one seems good but I can do it later
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB
# Bernoulli was best of above
import math
X_train = [one_hot(str(elem), 10000) for elem in X_train_base]
X_test =  [one_hot(str(elem), 10000) for elem in X_test_base]
X_train = pad_sequences(X_train, padding='post', maxlen=500)
X_test = pad_sequences(X_test, padding='post', maxlen=500)

svmClassifier = svm.SVC()
svmClassifier.fit(X_train, y_train_base)
svmPrediction = svmClassifier.predict(X_test)
svmPrediction = [math.floor(0.5+pred) for pred in svmPrediction]
print("CoAID_only: Results of SVM internal validation:")
print(classification_report(y_test_base, svmPrediction))

svmValidationPrediction = svmClassifier.predict(validationX)
svmValidationPrediction = [math.floor(0.5+pred) for pred in svmValidationPrediction]
print("CoAID_only: Results of SVM external validation, on our new dataset:")
print(classification_report(validationY, svmValidationPrediction))



X_train = [one_hot(str(elem), 10000) for elem in X_train_base]
X_test =  [one_hot(str(elem), 10000) for elem in X_test_base]
X_train = pad_sequences(X_train, padding='post', maxlen=500)
X_test = pad_sequences(X_test, padding='post', maxlen=500)

lrClassifier = LogisticRegression(random_state=0, max_iter=500)
lrClassifier.fit(X_train, y_train_base)
lrPrediction = lrClassifier.predict(X_test)
lrPrediction = [math.floor(0.5+pred) for pred in lrPrediction]
print("CoAID_only: Results of LR internal validation:")
print(classification_report(y_test_base, lrPrediction))

lrValidationPrediction = lrClassifier.predict(validationX)
lrValidationPrediction = [math.floor(0.5+pred) for pred in lrValidationPrediction]
print("CoAID_only: Results of LR external validation, on our new dataset:")
print(classification_report(validationY, lrValidationPrediction))

X_train = [one_hot(str(elem), 10000) for elem in X_train_base]
X_test =  [one_hot(str(elem), 10000) for elem in X_test_base]
X_train = pad_sequences(X_train, padding='post', maxlen=500)
X_test = pad_sequences(X_test, padding='post', maxlen=500)

bnbClassifier = BernoulliNB()
bnbClassifier.fit(X_train, y_train_base)
bnbPrediction = bnbClassifier.predict(X_test)
bnbPrediction = [math.floor(0.5+pred) for pred in bnbPrediction]
print("CoAID_only: Results of Bernoulli NB internal validation:")
print(classification_report(y_test_base, bnbPrediction))
bnbValidationPrediction = bnbClassifier.predict(validationX)
bnbValidationPrediction = [math.floor(0.5+pred) for pred in bnbValidationPrediction]
print("CoAID_only: Results of Bernoulli NB external validation, on our new dataset:")
print(classification_report(validationY, bnbValidationPrediction))



CoAID_only: Results of SVM internal validation:
              precision    recall  f1-score   support

           0       0.90      0.42      0.57       134
           1       0.59      0.95      0.72       116

    accuracy                           0.66       250
   macro avg       0.74      0.68      0.65       250
weighted avg       0.76      0.66      0.64       250

CoAID_only: Results of SVM external validation, on our new dataset:
              precision    recall  f1-score   support

           0       0.85      0.44      0.58      3883
           1       0.57      0.91      0.70      3175

    accuracy                           0.65      7058
   macro avg       0.71      0.67      0.64      7058
weighted avg       0.72      0.65      0.63      7058

CoAID_only: Results of LR internal validation:
              precision    recall  f1-score   support

           0       0.81      0.46      0.59       134
           1       0.58      0.87      0.70       116

    accuracy       